In [1]:
#AGAIN

In [2]:
# kill everying running on port 8000, kill any previous cloudflared's
!fuser -k 8000/tcp || true
!pkill -f cloudflared || true

^C


In [3]:
# Uninstall previous stuff if you need to. Sometimes need to turn it off and back on again

# These often pin incompatible Torch versions in Colab
!pip -q uninstall -y fastai torch torchvision torchaudio vllm xformers || true


In [4]:
# These pip installs are a lot, takes a while (like 7min+)

# PyTorch CUDA 12.1 wheels
!pip -q install --extra-index-url https://download.pytorch.org/whl/cu121 \
  "torch==2.4.1" "torchvision==0.19.1" "torchaudio==2.4.1"

# vLLM + helper libs
!pip -q install "vllm==0.10.0" huggingface_hub pycloudflared

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 799.0/799.0 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 33.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# 3) HF login (consider rotating your token later)
from huggingface_hub import login
login(token="HuggingFaceToken")

# 4) Start vLLM (T4-safe) and wait for /health
import os, subprocess, sys, time, requests
# T4-safe: no FA2, no xformers
os.environ["VLLM_USE_TRITON_FLASH_ATTN"] = "0"
os.environ["VLLM_USE_XFORMERS"] = "0" #Was having problems with xformers, so just dont use it?

In [6]:
cmd = [
  sys.executable, "-m", "vllm.entrypoints.openai.api_server",
  "--model", "hugging-quants/Meta-Llama-3.1-8B-Instruct-AWQ-INT4",
  "--quantization", "awq",
  "--max-model-len", "2048",
  "--max-num-seqs", "2",
  "--gpu-memory-utilization", "0.85",
  "--enforce-eager",
  "--host", "0.0.0.0",
  "--port", "8000",
]
logf = open("vllm.log", "wb")
proc = subprocess.Popen(cmd, stdout=logf, stderr=subprocess.STDOUT, start_new_session=True)
print("vLLM PID:", proc.pid)

# for i in range(90):
#     try:
#         r = requests.get("http://127.0.0.1:8000/health", timeout=1.5)
#         if r.ok and r.json().get("status") == "ok":
#             print("✅ vLLM is up on http://127.0.0.1:8000")
#             break
#     except Exception:
#         pass
#     if i % 5 == 0:
#         print(f"waiting for vLLM… ({i+1})")
#     time.sleep(2)
# else:
#     import subprocess as sp
#     print("❌ Timeout. Last 120 log lines:")
#     print(sp.run(["tail","-n","120","vllm.log"], text=True, capture_output=True).stdout)
#     raise SystemExit(1)

vLLM PID: 2182


In [ ]:
from pycloudflared import try_cloudflare
url = try_cloudflare(port=8000, verbose=True)
print("Public URL:", url)



# Keep the tunnel alive
import time
while True:
    time.sleep(60)

Download cloudflared...:   0%|          | 0/41181190 [00:00<?, ?it/s]

 * Running on https://port-better-operated-problems.trycloudflare.com
 * Traffic stats available on http://127.0.0.1:20241/metrics
Public URL: Urls(tunnel='https://port-better-operated-problems.trycloudflare.com', metrics='http://127.0.0.1:20241/metrics', process=<Popen: returncode: None args: ['/usr/local/lib/python3.11/dist-packages/pyc...>)
